# Demo for using `hifir4py` #
In this example, we show how to use `hifir4py` HIFIR preconditioner coupling with the built-in GMRES solver.. The example system is a saddle-point formulation of 3D Stokes equation with Taylor-Hood elements.

In [1]:
from hifir4py import *
import numpy as np

The matrix is stored by the HIFIR native binary format that is leading symmetric block aware. It's worht noting that. The following code shows how to load the matrix.

In [2]:
# load matrix and leading block size m
rowptr, colind, vals, shape, _ = read_hifir("demo_inputs/A.hilucsi")

Let's show some basic information of the system, including shape, nnz, and leading block symmetry

In [3]:
# A is scipy.sparse.csr_matrix
print("The system shape is {}, where the nnz is {}".format(rowptr[-1], shape))

The system shape is 44632, where the nnz is (2990, 2990)


The rhs vector can be directly loaded from `numpy` ASCII routine

In [4]:
b = np.loadtxt('demo_inputs/b.txt')

In [5]:
assert shape[0] == len(b)

Now, let's build the preconditioenr $\boldsymbol{M}$ with default configurations.

In [6]:
M = HIF()
M.factorize(rowptr, colind, vals, shape=shape)


|           Hybrid (Hierarchical) Incomplete Factorizations           |
|                                                                     |
| HIF is a package for computing hybrid (hierarchical) incomplete fa- |
| ctorizations with nearly linear time complexity.                    |
-----------------------------------------------------------------------

 > Package information:

		* Copyright (C) The HIF AUTHORS
		* Version: 1.0.0
		* Built on: 11:33:49, Jun  7 2021


Params (control parameters) are:

tau_L                         0.000100
tau_U                         0.000100
kappa_d                       3.000000
kappa                         3.000000
alpha_L                       10.000000
alpha_U                       10.000000
rho                           0.500000
c_d                           10.000000
c_h                           2.000000
N                             -1
verbose                       info
rf_par                        1
reorder                       Auto

With the preconditioenr successfully been built, let's print out some basic information

In [7]:
print('M levels are {}, with nnz {}'.format(M.levels, M.nnz))

M levels are 2, with nnz 141018


Now, we solve with the built-in flexible GMRES solver, with default configurations, i.e. restart is 30, relative convergence tolerance is 1e-6, and maximum allowed iterations are 500.

In [8]:
solver = GMRES(M)

In [9]:
x, iters = solver.solve(rowptr, colind, vals, b, shape=shape)

- GMRES -
rtol=1e-06
restart/cycle=30
maxiter=500
flex-kernel: tradition
init-guess: no

Calling traditional GMRES kernel...
Enter outer iteration 1...
  At iteration 1, relative residual is 5.01853e-06.
  At iteration 2, relative residual is 1.17415e-08.


In [10]:
print('solver done, with {} iterations and residule is {}'.format(iters, solver.resids[-1]))

solver done, with 2 iterations and residule is 1.3711755436588413e-07
